# **Power System Simulation Final Presentation FUSE**

## **Introduction**
Welcome to the final presentation on Power System Simulation. In this presentation, we will cover the following packages:

`graph_processing.py`
- Initialization and validation
- Find downstream vertices
- Find alternatie edges

`power_grid_modelling.py`
- Handle PGM input format
- Aggregate power flow results in tables
- Show tables


`grid_analytic.py`
- Calculate EV penetration levels
- Calculate optimal tap positions
- Do N-1 calculations

We have tested all the packages using `pytest` to ensure there are no errors and that the output is as expected. For performance checking, we will also run a timer to check how fast the package would run.

To be able to use the package, users must install the following dependencies (libraries):
* `numpy`
* `pandas`
* `networkx`
* `power-grid-model`
* `math`
* `random`

In [1]:
# Create a timer class
import time

class Timer:
    def __init__(self, name: str):
        self.name = name
        self.start = None

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, *args):
        print(
            f'Execution time for {self.name} is {(time.perf_counter() - self.start):0.6f} s'
        )

In [2]:
# Importing all functions from the package
from power_system_simulation.graph_processing import GraphProcessor
from power_system_simulation.power_grid_modelling import PowerGridModelling
from power_system_simulation.grid_analytic import GridAnalysis

## **1.Graph-Processing**

This section covers the functionalities and usage of the `graph_processing.py` package. The `graph_processing.py` package builds an undirected graph and implements two functionalties namely, `find_downstream_vertices()` and `find_alternative_edges()`.
This section covers the functionalities and usage of the `graph_processing.py` package. The `graph_processing.py` package builds an undirected graph and implements two functionalties namely, `find_downstream_vertices()` and `find_alternative_edges()`.

### **1.1 Input data**

To create a graph the following parameters need to be defined:
* `edge_ids`
* `edge_vertex_id_pairs`
* `edge_enabled`
* `source_vertex_id`

For example, we have a network created as follow:

      ------- edge_1---- source_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2 -- edge_7 -- vertex_4  -- edge_8 --vertex_6
      |
    edge_9
      |
    vertex_10  

If we only take into account the enabled edges:

      ------- edge_1---- source_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2            vertex_4                vertex_6
      |
    edge_9
      |
    vertex_10  

In [3]:
# Create input data
# List of edge IDs.    
edge_ids = [1, 3, 5, 7, 8, 9]
# Tuples representing connections between vertices, defining the graph's edges.
edge_vertex_id = [(0, 2), (0, 4), (0, 6), (2, 4), (4, 6), (2, 10)]
# List indicating whether each edge is enabled or disabled.
edge_enabled = [True, True, True, False, False, True]
# ID of the vertex that is also a source.
source_id = 0

# Calling the class
result = GraphProcessor(
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id,
    edge_enabled=edge_enabled,
    source_vertex_id=source_id,
)

If we want to know what is the downstream vertices at a given edge (for example edge 1) with respect to the source:

In [4]:
edge_id = 1

downstream_vertices = result.find_downstream_vertices(edge_id=edge_id)
print(downstream_vertices)

[2, 10]


And if we want to know the alternative adges to be connected if we disconnect the edge 5 

In [5]:
edge_id = 5

alternative_eges = result.find_alternative_edges(edge_id)
print(alternative_eges)

[8]


## **2 Power Grid Modelling Package**
In this assignment, we used the `power_grid_model` package as our core package for calculations.

We also used `pandas` and `numpy` for dataframes and ndarrays related functionality. For coherence with previous assignment, we also created a class called PowerGridModelling, along with some expected errors.
We assumed that the input data and the load profiles will be given in standard PGM format, therefore we expected users to have the input data as json file and load profile files as parquet files. From that, users can conveniently paste the paths of the file into our function.

In [6]:
import numpy as np
import pandas as pd
import json
import scipy

from power_grid_model import (
    LoadGenType,
    PowerGridModel,
    CalculationType,
    CalculationMethod,
    initialize_array
)

from power_grid_model.validation import (
    assert_valid_input_data,
    assert_valid_batch_data
)

from power_grid_model.utils import (
    json_deserialize
)

### **2.1 Input Data**
* A power grid in PGM input format
* A table containing active load profile of all the (`sym_load`) in the grid, with timestamps and load ids.
* A table containing reactive load profile of all the (`sym_load`) in the grid, with timestamps and load ids.




In [7]:
# Define file paths
data_path = "data/input_network_data.json"
active_load_profile_path = "data/active_power_profile.parquet"
reactive_load_profile_path = "data/reactive_power_profile.parquet"

# Load JSON data
with open(data_path) as fp:
    dataset = json.load(fp)

# Load Parquet data
active_load_profile = pd.read_parquet(active_load_profile_path)
reactive_load_profile = pd.read_parquet(reactive_load_profile_path)

The `dataset` encompasses multiple tables that detail various components of the electrical grid, including nodes, lines, sources, and load details.

In [8]:
node_data = dataset['data']['node']
df_node_data = pd.DataFrame(node_data)
display(df_node_data)

line_data = dataset['data']['line']
df_line_data = pd.DataFrame(line_data)
display(df_line_data)

sym_load_data = dataset['data']['sym_load']
df_sym_load_data = pd.DataFrame(sym_load_data)
display(df_sym_load_data)

source_data = dataset['data']['source']
df_source_data = pd.DataFrame(source_data)
display(df_source_data)

,id,u_rated
0,1,10500
1,2,10500
2,3,10500
3,4,10500


,id,from_node,to_node,from_status,to_status,r1,x1,c1,tan1,i_n
0,5,1,2,1,1,0.25,0.2,0.00001,0,1000
1,6,2,3,1,1,0.25,0.2,0.00001,0,1000
2,7,3,4,1,1,0.25,0.2,0.00001,0,1000


,id,node,status,type,p_specified,q_specified
0,8,2,1,0,0,0
1,9,3,1,0,0,0
2,10,4,1,0,0,0


,id,node,status,u_ref,sk
0,11,1,1,1,200000000


The `active_power_profile` and `reactive_power_profile` provide detailed timelines of power requirements for various loads, indicating the amount of power that needs to be supplied.

In [9]:
display(active_load_profile)
display(reactive_load_profile)

Load_ID,8,9,10
Timestamp,,,
2024-01-01 00:00:00,97627.007855,430378.732745,205526.752143
2024-01-01 01:00:00,89766.365994,-152690.401322,291788.226133
2024-01-01 02:00:00,-124825.577475,783546.001564,927325.521002
2024-01-01 03:00:00,-233116.962348,583450.076165,57789.839506
2024-01-01 04:00:00,136089.122188,851193.276585,-857927.883604
2024-01-01 05:00:00,-825741.400597,-959563.205119,665239.691096
2024-01-01 06:00:00,556313.501900,740024.296494,957236.684466
2024-01-01 07:00:00,598317.128433,-77041.275494,561058.352573
2024-01-01 08:00:00,-763451.148262,279842.042655,-713293.425182


Load_ID,8,9,10
Timestamp,,,
2024-01-01 00:00:00,-470888.775791,548467.378868,-87699.335567
2024-01-01 01:00:00,136867.897737,-962420.399127,235270.994152
2024-01-01 02:00:00,224191.445445,233867.993750,887496.157029
2024-01-01 03:00:00,363640.598207,-280984.198852,-125936.092401
2024-01-01 04:00:00,395262.391855,-879549.056741,333533.430891
2024-01-01 05:00:00,341275.739236,-579234.877852,-742147.404690
2024-01-01 06:00:00,-369143.298152,-272578.458115,140393.540836
2024-01-01 07:00:00,-122796.973075,976747.676118,-795910.378504
2024-01-01 08:00:00,-582246.487810,-677380.964230,306216.650931


### **2.2 Power Flow Results in Tables**

A table with each row representing a timestamp, with the following columns:
- Timestamp (index column)
- Maximum p.u. voltage of all the nodes for this timestamp
- The node ID with the maximum p.u. voltage
- Minimum p.u. voltage of all the nodes for this timestamp
- The node ID with the minimum p.u. voltage

A table with each row representing a line, with the following columns:
- Line ID (index column)
- Energy loss of the line across the timeline in kWh
- Maximum loading in p.u. of the line across the whole timeline
- Timestamp of this maximum loading moment
- Minimum loading in p.u. of the line across the whole timeline
- Timestamp of this minimum loading moment

In [10]:
# Initialize the PowerGridModelling class
pgm = PowerGridModelling(data_path, active_load_profile_path, reactive_load_profile_path)

df_per_timestamp = pgm.data_per_timestamp()
df_per_line = pgm.data_per_line()

# Display the results
print("\nData Per Timestamp:")
display(df_per_timestamp.head())

print("\nData Per Line:")
display(df_per_line.head())
# display(DataFrame(df_per_line.head()))


Data Per Timestamp:


,Max_Voltage,Max_Voltage_Node,Min_Voltage,Min_Voltage_Node
Timestamp,,,,
2024-01-01 00:00:00,1.004847,1,1.003450,3
2024-01-01 01:00:00,1.012053,3,1.007998,1
2024-01-01 02:00:00,0.997474,1,0.984365,4
2024-01-01 03:00:00,1.006557,4,1.005190,1
2024-01-01 04:00:00,1.011007,4,1.005877,1



Data Per Line:


,Total_Loss,Max_loading,Max_Loading_Timestamp,Min_Loading,Min_Loading_Timestamp
Line_ID,,,,,
5,63.294763,0.149830,2024-01-01 06:00:00,0.063798,2024-01-01 03:00:00
6,36.775143,0.111039,2024-01-01 05:00:00,0.037184,2024-01-01 00:00:00
7,14.872359,0.071700,2024-01-01 02:00:00,0.020380,2024-01-01 01:00:00


## **3. Grid Analysis**
In this part a package with some low voltage (LV) functions is presented. A LV grid with one transformer, one source and many `sym_load` representing houses is condiderd in this part.

### **3.1 input data**
The class called `GridAnalysis` is used for this package, it takes these arguments:
* Path of LV grid network data (`data_path`)
* List of LV feeder IDs (`feeder_ids`)
* Path of active load profile (`active_load_profile_path`)
* Path of reactive load profile (`reactive_load_profile_path`)
* Path of pool of EV charging profiles (`ev_pool_path`)

The input data is loaded as follows:

In [11]:
data_path="data/big_network/input_network_data.json"
feeder_ids=[1204,1304,1404,1504,1604,1704,1804,1904]
active_load_profile_path="data/big_network/active_power_profile.parquet"
reactive_load_profile_path="data/big_network/reactive_power_profile.parquet"
ev_pool_path="data/big_network/ev_active_power_profile.parquet"

data = [data_path, active_load_profile_path, reactive_load_profile_path, ev_pool_path]

with Timer("data loading"):
    data = GridAnalysis(data, feeder_ids)

Execution time for data loading is 0.000004 s


The input data is checked for the following validity criteria:

1. checks done in this part:
  * The LV grid has exactly one transformer, and one source.
  * All IDs in the LV Feeder IDs are valid line IDs.
  * All the lines in the LV Feeder IDs have the from_node the same as the to_node of the transformer.
  * The timestamps are matching between the active load profile, reactive load profile, and EV charging profile.
  * The number of EV charging profile is at least the same as the number of sym_load.

2. checks done in first part:
  * The grid is fully connected in the initial state.
  * The grid has no cycles in the initial state.

3. checks done in second part:
  * The LV grid should be a valid PGM input data.
  * The IDs in active load profile and reactive load profile are matching.
  * The IDs in active load profile and reactive load profile are valid IDs of sym_load.

### **3.2 EV penetration level**

The EV penetration level function adds EV charging profiles to a percentage of houses which has EV charged at home (EV penetration level). The houses and charging profiles are chosen randomly. This is done as follows:

1. The number of EVs per LV feeder are calculated using: `[penetration_level * total_houses / number_of_feeders]`
2. The houses attached to each LV feeder are indentified using the function `find_downstream_vertices` from part 1
3. For each LV feeder a random charging profile is added to the specified number of random houses
4. The function `powerGridModeling` from part 2 is than called to get the new results

In [12]:
penetration_level = 0.2

with Timer("EV penetration level"):
  df_result_per_timestamp, df_result_per_line = data.ev_penetration_level(penetration_level)

print("\nData Per Timestamp:")
print(df_result_per_timestamp)
print("\nData Per Line:")
print(df_result_per_line)

Execution time for EV penetration level is 185.770563 s
u_pu per timestamp results:
                     Max_Voltage  Max_Voltage_Node  Min_Voltage  \
Timestamp                                                         
2025-01-01 00:00:00     1.074658                 1     1.049898   
2025-01-01 00:15:00     1.074532                 1     1.049891   
2025-01-01 00:30:00     1.074927                 1     1.049916   
2025-01-01 00:45:00     1.074601                 1     1.049894   
2025-01-01 01:00:00     1.074482                 1     1.049885   
...                          ...               ...          ...   
2025-12-31 22:45:00     1.070122                 1     1.049706   
2025-12-31 23:00:00     1.070500                 1     1.049730   
2025-12-31 23:15:00     1.070524                 1     1.049718   
2025-12-31 23:30:00     1.070258                 1     1.049710   
2025-12-31 23:45:00     1.069437                 1     1.049665   

                     Min_Voltage_Node  
Time

### **3.3 N-1 calculation**

In [13]:
edge_id = 1500

with Timer("N-1 calculation"):
  result = data.alternative_grid_topology(edge_id=edge_id)

print(result)

Line ID provided is not in line IDs.
Execution time for N-1 calculation is 0.000109 s


IDNotFoundError: Line ID provided is not in line IDs.